In [7]:
# import pandas_ta as ta
import ta
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import main_functions
import plotly.graph_objects as go
import warnings
from binance.client import Client   # Load historical price data from binance exchange 
client = Client()

warnings.filterwarnings("ignore", category=FutureWarning)

In [8]:
import importlib
# If you want to update the module:
importlib.reload(main_functions)
from main_functions import *

In [120]:
pd.set_option('display.max_rows', 400)  # Set the maximum number of rows to display

# Defining the variables

In [94]:
ticker = "BTCUSDT"
# timeframe = "1h"
# start = "2023-01-01"
timeframe = "1d"
start = "2019-09-08"
start_date = datetime.strptime(start, "%Y-%m-%d")
end = str(datetime.now())
# Calculate the difference in days
day = (datetime.now() - start_date).days

# OTHER PARAMETERS
fast_ma_period=20
slow_ma_period=50
rsi_period=14
rsi_value=50

sl_perc= 5
tp_perc= 10

In [95]:
df = getdata(ticker, timeframe, day)
df

,Open,High,Low,Close,Volume
timestamp,,,,,
2019-09-08,10000.00,10412.65,10000.00,10391.63,3096.291
2019-09-09,10316.62,10475.54,10077.22,10307.00,14824.373
2019-09-10,10307.00,10382.97,9940.87,10102.02,9068.955
2019-09-11,10094.27,10293.11,9884.31,10159.55,10897.922
2019-09-12,10163.06,10450.13,10042.12,10415.13,15609.634
...,...,...,...,...,...
2023-10-24,33074.00,35300.00,32833.50,33917.00,921993.063
2023-10-25,33917.00,35154.30,33660.60,34487.70,465367.144
2023-10-26,34487.80,34828.70,33750.00,34154.00,339552.964


In [121]:
# Define the function for the trading strategy
def getSignals(ticker, timeframe, day, fast_ma_period=20, slow_ma_period=50, rsi_period=14, rsi_value=50):
    # df = getdata(ticker, timeframe)  # Assuming you have a getdata function that fetches the data

    # Calculate the moving averages
    df['shifted_open'] = df.Open.shift(-1)
    df['Fast_MA'] = ta.trend.sma_indicator(df['Close'], window=fast_ma_period)
    df['Slow_MA'] = ta.trend.sma_indicator(df['Close'], window=slow_ma_period)
    # Calculate the RSI
    rsi = ta.momentum.RSIIndicator(df['Close'], window=rsi_period)
    df['RSI'] = rsi.rsi()

    df['buySignal'] = np.where(
        (df['Fast_MA'] > df['Slow_MA']) &
        (df['Fast_MA'].shift(1) <= df['Slow_MA'].shift(1)) &
        (df['RSI'] > rsi_value),
        1,
        0
    )
    
    return df

In [167]:
df = getSignals(df, fast_ma_period=fast_ma_period, slow_ma_period=slow_ma_period, rsi_period=rsi_period, rsi_value=rsi_value)
df

,Open,High,Low,Close,Volume,shifted_open,Fast_MA,Slow_MA,RSI,buySignal
timestamp,,,,,,,,,,
2019-09-08,10000.00,10412.65,10000.00,10391.63,3096.291,10316.62,NaN,NaN,NaN,0
2019-09-09,10316.62,10475.54,10077.22,10307.00,14824.373,10307.00,NaN,NaN,NaN,0
2019-09-10,10307.00,10382.97,9940.87,10102.02,9068.955,10094.27,NaN,NaN,NaN,0
2019-09-11,10094.27,10293.11,9884.31,10159.55,10897.922,10163.06,NaN,NaN,NaN,0
2019-09-12,10163.06,10450.13,10042.12,10415.13,15609.634,10414.96,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...
2023-10-23,29981.40,35981.90,29869.40,33074.00,987741.536,33074.00,28239.435,27177.836,85.330772,0
2023-10-24,33074.00,35300.00,32833.50,33917.00,921993.063,33917.00,28547.295,27339.896,86.952942,0
2023-10-25,33917.00,35154.30,33660.60,34487.70,465367.144,34487.80,28901.760,27514.056,87.926345,0


In [168]:
# Assuming you have a DataFrame 'df' and want to find rows where 'buySignal' is equal to 1
buy_signal_rows = df[df['buySignal'] == 1]
buy_signal_rows

,Open,High,Low,Close,Volume,shifted_open,Fast_MA,Slow_MA,RSI,buySignal
timestamp,,,,,,,,,,
2019-11-05,9395.47,9457.13,9164.33,9313.20,113738.226,9313.10,8703.3025,8682.6032,58.971389,1
2020-01-08,8150.90,8468.42,7870.11,8059.84,321225.114,8059.77,7366.6190,7327.0674,65.355463,1
2020-04-19,7242.49,7258.42,7055.00,7118.03,216975.114,7118.04,6938.6770,6901.2634,53.523251,1
2020-07-27,9939.68,11488.00,9928.61,11047.58,744752.400,11047.58,9437.6935,9382.6004,82.893621,1
2020-10-13,11530.37,11562.00,11308.00,11419.36,199115.115,11419.36,10860.0840,10833.4172,63.064975,1
2021-08-01,41448.10,42614.98,39350.40,39846.78,487529.044,39846.78,35021.1210,34921.0338,63.349047,1
2021-10-11,54700.00,57880.00,54458.00,57500.01,314868.050,57500.01,48011.8605,47737.2122,71.347360,1
2022-03-19,41751.60,42446.60,41500.00,42188.20,186459.954,42188.20,40446.2150,40406.6474,56.516074,1
2022-03-27,44489.30,47047.00,44416.00,46820.80,269280.228,46820.90,41360.3500,41142.4184,68.662433,1


# Backtest

In [118]:
def backtest(ticker, timeframe, day, fast_ma_period=20, slow_ma_period=50, rsi_period=14, rsi_value=50, sl_perc=sl_perc*0.01, tp_perc=tp_perc*0.01):
    print("fast_ma_period", fast_ma_period)
    print("slow_ma_period", slow_ma_period)
    print("rsi_period", rsi_period)
    print("sl_perc", sl_perc)
    print("tp_perc", tp_perc)
    print("rsi_value", rsi_value)
    df = getSignals(ticker, timeframe, day, fast_ma_period=fast_ma_period, slow_ma_period=slow_ma_period, rsi_period=rsi_period, rsi_value=rsi_value)  # Use the trading_strategy function to get signals

    in_position = False
    buy_pos = False
    sell_pos = False

    results_df = pd.DataFrame(columns=['ticker', 'returns', 'winrate', 'trades', 'buy&hold_ret%'])
    buydates, buyprices = [], []
    selldates, sellprices = [], []
    profits = []
    sl_list = []  # List to store stop_loss values
    tp_list = []  # List to store target_price values
    
    for index, row in df.iterrows():
        # long position block
        if not in_position and row['buySignal'] == 1:
            buyprice = row['Close']
            buydates.append(index)
            buyprices.append(buyprice)
            in_position = True
            buy_pos = True
            stop_loss = buyprice * (1 - sl_perc)
            target_price = buyprice * (1 + tp_perc)
            sl_list.append(stop_loss)  # Store stop_loss value
            tp_list.append(target_price)  # Store target_price value

        elif in_position and buy_pos:
            if row['Low'] <= stop_loss:
                selldates.append(index)
                sellprice = stop_loss
                sellprices.append(sellprice)
                in_position = False
                buy_pos = False
                profits.append((sellprice - buyprice) / buyprice - 0.001)  # Account for a small commission fee (0.1%)

            elif row['High'] >= target_price:
                selldates.append(index)
                sellprice = target_price
                sellprices.append(sellprice)
                in_position = False
                buy_pos = False
                profits.append((sellprice - buyprice) / buyprice - 0.001)  # Account for a small commission fee (0.1%)

    try:
        if len(buydates) == 0:
            print("No trades were made.")
        else:
            returns = (pd.Series(profits, dtype=float) + 1).prod() - 1
            returns = round(returns * 100, 2)

            wins = sum(1 for i in profits if i > 0)
            winrate = round((wins / len(buydates)) * 100, 2)
            ct = min(len(buydates), len(selldates))
            buy_hold_ret = (df['Close'][-1] - df['Open'][0]) / df['Open'][0] * 100
            # Calculate your custom score here (e.g., a weighted combination of returns and win rate)
            custom_score = returns * (winrate / 100)
            
            results_df.loc[len(results_df)] = [ticker, returns, winrate, ct, buy_hold_ret]
            print(f'{ticker}, winrate={winrate}%, returns={returns}%, no. of trades = {ct}, buy&hold_ret = {buy_hold_ret}%')
    except:
        print('Invalid input')

    return {
        'buydates': buydates,
        'buyprices': buyprices,
        'selldates': selldates,
        'sellprices': sellprices,
        'profits': profits,
        'stop_loss': sl_list,  # Include stop_loss values
        'take_profit': tp_list,  # Include target_price values
        # Other results...
    }, results_df, df, custom_score

def displayTrades(**kwargs):
    # Access the trade data and other results from kwargs
    buydates = kwargs['buydates']
    buyprices = kwargs['buyprices']
    selldates = kwargs['selldates']
    sellprices = kwargs['sellprices']
    profits = kwargs['profits']
    stop_loss = kwargs['stop_loss']
    take_profit = kwargs['take_profit']
    
    ct = min(len(buydates),len(selldates))
    dfr =pd.DataFrame()
    dfr['buydates']= buydates[:ct]
    dfr['buyprice']= buyprices[:ct]
    dfr['selldates'] = selldates[:ct]
    dfr['sellprice'] = sellprices[:ct]
    dfr['profits'] = (profits[:ct])
    dfr['commulative_returns'] = ((pd.Series(profits) + 1).cumprod())
    dfr['tradeSide'] = np.where(dfr['buydates'] < dfr['selldates'], 'Long', 'Short')
    
    dfr['entry_date'] = np.where(dfr['tradeSide'] == 'Long', dfr['buydates'], dfr['selldates'])
    dfr['exit_date'] = np.where(dfr['tradeSide'] == 'Long', dfr['selldates'], dfr['buydates'])
    dfr['entry_price'] = np.where(dfr['tradeSide'] == 'Long', dfr['buyprice'], dfr['sellprice'])
    dfr['exit_price'] = np.where(dfr['tradeSide'] == 'Long', dfr['sellprice'], dfr['buyprice'])
    
    # Create a list of column names with the desired display order
    display_order = ['entry_date', 'entry_price', 'exit_date', 'exit_price', 'profits', 'commulative_returns', 'tradeSide']
    
    # Create a copy of the DataFrame with only the columns to be displayed
    dfr_display = dfr[display_order].copy()
    dfr_display['stop_loss'] = (stop_loss[:ct])
    dfr_display['take_profit'] = (take_profit[:ct])
    
    return dfr, dfr_display

In [122]:
results_data, results_df, df, score = backtest(ticker, timeframe, day, fast_ma_period, slow_ma_period, rsi_period, rsi_value, sl_perc*0.01, tp_perc*0.01)

fast_ma_period 20
slow_ma_period 50
rsi_period 14
sl_perc 0.05
tp_perc 0.1
rsi_value 50
BTCUSDT, winrate=28.57%, returns=-13.57%, no. of trades = 14, buy&hold_ret = 241.39699999999996%


In [123]:
results_df

,ticker,returns,winrate,trades,buy&hold_ret%
0,BTCUSDT,-13.57,28.57,14,241.397


In [124]:
score

-3.876949

In [125]:
dfr, dfr_display = displayTrades(**results_data)
dfr_display

,entry_date,entry_price,exit_date,exit_price,profits,commulative_returns,tradeSide,stop_loss,take_profit
0,2019-11-05,9313.20,2019-11-08,8847.5400,-0.051,0.949000,Long,8847.5400,10244.520
1,2020-01-08,8059.84,2020-01-14,8865.8240,0.099,1.042951,Long,7656.8480,8865.824
2,2020-04-19,7118.03,2020-04-20,6762.1285,-0.051,0.989760,Long,6762.1285,7829.833
3,2020-07-27,11047.58,2020-08-02,10495.2010,-0.051,0.939283,Long,10495.2010,12152.338
4,2020-10-13,11419.36,2020-10-21,12561.2960,0.099,1.032272,Long,10848.3920,12561.296
5,2021-08-01,39846.78,2021-08-03,37854.4410,-0.051,0.979626,Long,37854.4410,43831.458
6,2021-10-11,57500.01,2021-10-12,54625.0095,-0.051,0.929665,Long,54625.0095,63250.011
7,2022-03-19,42188.20,2022-03-27,46407.0200,0.099,1.021702,Long,40078.7900,46407.020
8,2022-07-28,23836.20,2022-08-02,22644.3900,-0.051,0.969595,Long,22644.3900,26219.820
9,2022-10-31,20482.00,2022-11-08,19457.9000,-0.051,0.920146,Long,19457.9000,22530.200


In [77]:
# Create a subplot with two rows and one column
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02, row_width=[1, 2])

# Determine the desired y-axis range based on your data
y_min = df['Low'].min()  # Replace 'Low' with the appropriate column name
y_max = df['High'].max()  # Replace 'High' with the appropriate column name

bull_candle = '#e7bf4f'
bearish_candle = '#497ad2'

# Add candlestick trace to the main subplot
fig.add_trace(go.Candlestick(
    x=df.index,
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Close'],
    increasing_line_color=bull_candle,  # Bull candle outline color
    decreasing_line_color=bearish_candle,    # Bear candle outline color
    increasing_fillcolor=bull_candle,  # Bull candle fill color
    decreasing_fillcolor=bearish_candle,  # Bear candle fill color
    name='Candles',
), row=1, col=1)

# Add Fast_MA and Slow_MA to the main subplot (overlay)
fig.add_trace(go.Scatter(
    x=df.index,
    y=df['Fast_MA'],
    mode='lines',
    line=dict(color='blue'),
    name='SMA_FAST'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=df.index,
    y=df['Slow_MA'],
    mode='lines',
    line=dict(color='red'),
    name='SMA_SLOW'
), row=1, col=1)

entry_dates = {"Long": [], "Short": []}

for index, row in dfr.iterrows():
    if row["tradeSide"] == "Long":
        entry_dates["Long"].append(row['buydates'])
    else:
        entry_dates["Short"].append(row['selldates'])

# Plot the Long signals as green arrows
fig.add_trace(go.Scatter(
    x=entry_dates["Long"],
    y=df['Close'].loc[entry_dates["Long"]],
    mode='markers',
    marker=dict(
        symbol='arrow-bar-right',
        size=15,
        color='green',
    ),
    name='Long Signals',
))

# Plot the Short signals as red arrows
fig.add_trace(go.Scatter(
    x=entry_dates["Short"],
    y=df['Close'].loc[entry_dates["Short"]],
    mode='markers',
    marker=dict(
        symbol='arrow-bar-right',
        size=15,
        color='red',
    ),
    name='Short Signals',
))

# Plot stop_loss and target_price polygons
for index, row in dfr_display.iterrows():
    if not pd.isna(row['stop_loss']) and not pd.isna(row['take_profit']):
        fig.add_shape(
            type="rect",
            x0=row['entry_date'],
            x1=row['exit_date'],
            y0=row['stop_loss'],
            y1=row['entry_price'],
            fillcolor="rgba(255, 0, 0, 0.3)",  # Red polygon for stop_loss
            line=dict(width=0),
            xref="x",
            yref="y",
        )
        
        fig.add_shape(
            type="rect",
            x0=row['entry_date'],
            x1=row['exit_date'],
            y0=row['entry_price'],
            y1=row['take_profit'],
            fillcolor="rgba(0, 255, 0, 0.3)",  # Green polygon for target_price
            line=dict(width=0),
            xref="x",
            yref="y",
        )

# Customize the layout
# Update layout to customize the appearance (optional)
fig.update_layout(
    yaxis=dict(
        # Remove the 'range' parameter to let Plotly automatically determine the y-axis range
        range=[y_min, y_max],  # adjust scaling at y-axis
        # autorange=True,  # Set autorange to True to enable autoscaling
    ),
    height=800,  # Set the desired height in pixels
    title_text='Candlestick Chart with SMA and RSI',
    xaxis_rangeslider_visible=False,
    xaxis_title='Date',
    yaxis_title='Price',
    showlegend=True,
    hovermode='x'
)

# Set the hovermode to 'x unified' to enable a vertical line on hover
fig.update_layout(hovermode='x unified')

# Add RSI to a separate subplot
fig.add_trace(go.Scatter(
    x=df.index,
    y=df['RSI'],
    mode='lines',
    line=dict(color='green'),
    name='RSI'
), row=2, col=1)

# Add RSI upper, middle, and lower bands
fig.add_shape(
    type="line",
    x0=df.index.min(),
    x1=df.index.max(),
    y0=70,
    y1=70,
    line=dict(color="purple", width=1, dash="dot"),
    xref="x",
    yref="y",
    row=2,
    col=1
)

fig.add_shape(
    type="line",
    x0=df.index.min(),
    x1=df.index.max(),
    y0=50,
    y1=50,
    line=dict(color="purple", width=1, dash="dot"),
    xref="x",
    yref="y",
    row=2,
    col=1
)

fig.add_shape(
    type="line",
    x0=df.index.min(),
    x1=df.index.max(),
    y0=30,
    y1=30,
    line=dict(color="purple", width=1, dash="dot"),
    xref="x",
    yref="y",
    row=2,
    col=1
)

# Add a filled area between the bands
fig.add_shape(
    type="rect",
    x0=df.index.min(),
    x1=df.index.max(),
    y0=70,
    y1=30,
    fillcolor="rgba(128, 0, 128, 0.2)",  # Light purple color
    line=dict(width=0),
    xref="x",
    yref="y",
    row=2,
    col=1
)

# Add horizontal line for the crosshair in the subplot
fig.add_shape(
    type='line',
    x0=df.index[0],
    x1=df.index[-1],
    y0=1,
    y1=4,
    line=dict(color='purple', width=1, dash='dot'),
)


# Show the Plotly figure in the Jupyter Notebook
fig.show()

In [ ]:
import matplotlib.pyplot as plt

# Set the plot size
plt.figure(figsize=(10,6))

# Plot the account balance column
plt.plot(dfr.buydates, dfr.commulative_returns)

# Add labels to the plot
plt.title('Account Returns Over Time')
plt.xlabel('Time')
plt.ylabel('Returns')

# Display the plot
plt.show()

# Set Final Bot Parameters

In [131]:
ticker = "BTCUSDT"
timeframe = "1d"        # ----> TO BE OPTIMIZED
usdt_amount = 15


fast_ma_period=18           # ----> TO BE OPTIMIZED
slow_ma_period=75           # ----> TO BE OPTIMIZED
rsi_period=14               # ----> TO BE OPTIMIZED
rsi_value=50                # ----> TO BE OPTIMIZED
sl_perc=5                   # ----> TO BE OPTIMIZED
tp_perc=10                  # ----> TO BE OPTIMIZED

# Optuna Hyperparameter Optimization

In [ ]:
import optuna # !pip install optuna

# streamlit inputs should be fast_ma_period, slow_ma_period, rsi_period, rsi_value, sl_perc and tp_perc values also number of trials (default=100)

def objective(trial):
    fast_ma_period = trial.suggest_int("fast_ma_period", 15, 30)
    slow_ma_period = trial.suggest_int("slow_ma_period", 50, 75)
    rsi_period = trial.suggest_int("rsi_period", 11, 24)
    rsi_value = trial.suggest_int("rsi_value", 45, 60)
    sl_perc = trial.suggest_int("sl_perc", 3, 10)
    tp_perc = trial.suggest_int("tp_perc", 5, 15)
    
    # Run your strategy with the trial parameters and calculate a score
    _, __, ___, score = backtest(ticker, timeframe, day, fast_ma_period, slow_ma_period, rsi_period, sl_perc=sl_perc*0.01, tp_perc=tp_perc*0.01)
    
    return score

# Create an Optuna study and run optimization
study = optuna.create_study(direction="maximize")  # You can also use "minimize" depending on your objective
study.optimize(objective, n_trials=100)

# Retrieve the best parameters and score
best_params = study.best_params
best_score = study.best_value

print("Best Parameters:", best_params)
print("Best Score:", best_score)


In [127]:
best_params

{'fast_ma_period': 21,
 'slow_ma_period': 60,
 'rsi_period': 22,
 'rsi_value': 56,
 'sl_perc': 7,
 'tp_perc': 14}

In [134]:
# Fetch all tickers for the spot market
spot_tickers = client.get_all_tickers()


In [141]:
# Initialize an empty list to store the filtered symbols
binance_spot_usdt = []
# Loop through the tickers and filter only those ending with "USDT"
for ticker in spot_tickers:
    symbol = ticker['symbol']
    price = ticker['price']
    if symbol.endswith('USDT'):
        binance_spot_usdt.append(symbol)
    print(f"Symbol: {symbol}, Price: {price}")


Symbol: ETHBTC, Price: 0.05207000
Symbol: LTCBTC, Price: 0.00198300
Symbol: BNBBTC, Price: 0.00661400
Symbol: NEOBTC, Price: 0.00027420
Symbol: QTUMETH, Price: 0.00178100
Symbol: EOSETH, Price: 0.00035210
Symbol: SNTETH, Price: 0.00001638
Symbol: BNTETH, Price: 0.00030360
Symbol: BCCBTC, Price: 0.07908100
Symbol: GASBTC, Price: 0.00015450
Symbol: BNBETH, Price: 0.12700000
Symbol: BTCUSDT, Price: 34175.80000000
Symbol: ETHUSDT, Price: 1779.35000000
Symbol: HSRBTC, Price: 0.00041400
Symbol: OAXETH, Price: 0.00017780
Symbol: DNTETH, Price: 0.00002801
Symbol: MCOETH, Price: 0.00577200
Symbol: ICNETH, Price: 0.00166300
Symbol: MCOBTC, Price: 0.00021140
Symbol: WTCBTC, Price: 0.00000475
Symbol: WTCETH, Price: 0.00023700
Symbol: LRCBTC, Price: 0.00000564
Symbol: LRCETH, Price: 0.00010794
Symbol: QTUMBTC, Price: 0.00009230
Symbol: YOYOBTC, Price: 0.00000008
Symbol: OMGBTC, Price: 0.00003080
Symbol: OMGETH, Price: 0.00079100
Symbol: ZRXBTC, Price: 0.00000703
Symbol: ZRXETH, Price: 0.00009940
Sy

In [144]:
print(binance_spot_usdt)

['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'BCCUSDT', 'NEOUSDT', 'LTCUSDT', 'QTUMUSDT', 'ADAUSDT', 'XRPUSDT', 'EOSUSDT', 'TUSDUSDT', 'IOTAUSDT', 'XLMUSDT', 'ONTUSDT', 'TRXUSDT', 'ETCUSDT', 'ICXUSDT', 'VENUSDT', 'NULSUSDT', 'VETUSDT', 'PAXUSDT', 'BCHABCUSDT', 'BCHSVUSDT', 'USDCUSDT', 'LINKUSDT', 'WAVESUSDT', 'BTTUSDT', 'USDSUSDT', 'ONGUSDT', 'HOTUSDT', 'ZILUSDT', 'ZRXUSDT', 'FETUSDT', 'BATUSDT', 'XMRUSDT', 'ZECUSDT', 'IOSTUSDT', 'CELRUSDT', 'DASHUSDT', 'NANOUSDT', 'OMGUSDT', 'THETAUSDT', 'ENJUSDT', 'MITHUSDT', 'MATICUSDT', 'ATOMUSDT', 'TFUELUSDT', 'ONEUSDT', 'FTMUSDT', 'ALGOUSDT', 'USDSBUSDT', 'GTOUSDT', 'ERDUSDT', 'DOGEUSDT', 'DUSKUSDT', 'ANKRUSDT', 'WINUSDT', 'COSUSDT', 'NPXSUSDT', 'COCOSUSDT', 'MTLUSDT', 'TOMOUSDT', 'PERLUSDT', 'DENTUSDT', 'MFTUSDT', 'KEYUSDT', 'STORMUSDT', 'DOCKUSDT', 'WANUSDT', 'FUNUSDT', 'CVCUSDT', 'CHZUSDT', 'BANDUSDT', 'BUSDUSDT', 'BEAMUSDT', 'XTZUSDT', 'RENUSDT', 'RVNUSDT', 'HCUSDT', 'HBARUSDT', 'NKNUSDT', 'STXUSDT', 'KAVAUSDT', 'ARPAUSDT', 'IOTXUSDT', 'RLCUSDT', 'MCOUS

In [10]:
from binance.client import Client
from binance.enums import HistoricalKlinesType  # Import HistoricalKlinesType from binance.enums
import botConfig

In [11]:
# If you want to update the module:
importlib.reload(botConfig)
from botConfig import *

In [12]:
mode

'Sandbox/Demo'

In [24]:
def binanceActive():
    exchange = ccxt.binance({
        'apiKey': demo_apiKey,
        'secret': demo_secret,
        'enableRateLimit': True,
        'rateLimit': 10000,
        'options': {
            # 'recvWindow': 9000,  # replace with your desired recv_window value
            'test': True,  # use testnet (sandbox) environment
            # 'adjustForTimeDifference': True,
        }
    })
    exchange.set_sandbox_mode(enable=True)
    return exchange

exchange = binanceActive()
# print("Key: ", apiKey)
# print("Secret: ", secret)

In [29]:
balance = exchange.fetch_balance()['free']
balance

{'ETH': 1.0,
 'BTC': 1.0,
 'LTC': 7.0,
 'BNB': 2.0,
 'USDT': 10000.0,
 'TRX': 5104.0,
 'XRP': 829.0,
 'BUSD': 10000.0,
 'NEO': 53.0,
 'QTUM': 161.0,
 'EOS': 805.0,
 'SNT': 8727.0,
 'BNT': 951.0,
 'GAS': 78.0,
 'WTC': 3166.0,
 'LRC': 2528.0,
 'ZRX': 1875.0,
 'KNC': 633.0,
 'FUN': 18446.0,
 'IOTA': 3206.0,
 'LINK': 44.0,
 'XVG': 18446.0,
 'MTL': 323.0,
 'ETC': 28.0,
 'ZEC': 18.0,
 'AST': 4777.0,
 'DASH': 17.0,
 'OAX': 2758.0,
 'REQ': 6783.0,
 'VIB': 7532.0,
 'POWR': 2174.0,
 'ENJ': 1998.0,
 'STORJ': 1202.0,
 'KMD': 2146.0,
 'NULS': 2612.0,
 'XMR': 2.0,
 'AMB': 18446.0,
 'BAT': 2426.0,
 'LSK': 512.0,
 'MANA': 1400.0,
 'ADX': 3375.0,
 'ADA': 1740.0,
 'XLM': 4116.0,
 'WAVES': 286.0,
 'ICX': 2300.0,
 'ELF': 1399.0,
 'RLC': 452.0,
 'PIVX': 18466.0,
 'IOST': 18446.0,
 'STEEM': 2228.0,
 'BLZ': 1678.0,
 'ZIL': 18446.0,
 'ONT': 2393.0,
 'WAN': 2550.0,
 'SYS': 4803.0,
 'LOOM': 4309.0,
 'TUSD': 10000.0,
 'ZEN': 56.0,
 'THETA': 659.0,
 'IOTX': 18446.0,
 'QKC': 18446.0,
 'DATA': 18446.0,
 'SC': 18446

In [17]:
def get_spot_data(ticker, interval, start_str, end_str):
    # Initialize the Binance client
    client = Client()

    # Fetch historical spot klines from Binance
    klines = client.get_historical_klines(symbol=ticker, interval=interval, start_str=start_str, end_str=end_str, klines_type=HistoricalKlinesType.SPOT)
    # Convert the fetched data to a DataFrame
    df = pd.DataFrame(klines, columns=['timestamp', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close_time', 'Quote_asset_volume', 'Number_of_trades', 'Taker_buy_base_asset_volume', 'Taker_buy_quote_asset_volume', 'Ignore'])
    # Set the 'timestamp' as the index
    df.set_index('timestamp', inplace=True)
    # Convert the 'timestamp' index to datetime
    df.index = pd.to_datetime(df.index, unit='ms')
    # Convert columns to numeric
    df = df.apply(pd.to_numeric, errors='ignore')

    return df[['Open', 'High', 'Low', 'Close', 'Volume']]

In [18]:
# Example usage:
ticker = "BTCUSDT"
interval = "1d"
# Define the end date as today
end_date = datetime.now()
# Calculate the start date by subtracting 1000 days from the end date
start_date = end_date - timedelta(days=1000)

# Format the start and end dates as strings
start_str = start_date.strftime("%Y-%m-%d")
end_str = end_date.strftime("%Y-%m-%d")
print("Start Date", start_str)
print("End Date", end_str)
# start_str = "2023-01-01"
# end_str = "2023-10-26"

Start Date 2021-02-02
End Date 2023-10-30


In [19]:
spot_data = get_spot_data(ticker, interval, start_str, end_str)
spot_data

,Open,High,Low,Close,Volume
timestamp,,,,,
2021-02-02,33517.09,35984.33,33418.00,35466.24,78056.659880
2021-02-03,35472.71,37662.63,35362.38,37618.87,80784.333663
2021-02-04,37620.26,38708.27,36161.95,36936.66,92080.735898
2021-02-05,36936.65,38310.12,36570.00,38290.24,66681.334275
2021-02-06,38289.32,40955.51,38215.94,39186.94,98757.311183
...,...,...,...,...,...
2023-10-25,33922.73,35132.85,33679.05,34496.05,54887.025290
2023-10-26,34496.05,34824.13,33751.00,34151.66,39744.662550
2023-10-27,34151.66,34245.00,33390.95,33892.02,32330.401060


In [157]:
ticker = "BTCUSDT"
timeframe = "1d"
start = "2019-09-08"
start_date = datetime.strptime(start, "%Y-%m-%d")
end = str(datetime.now())
# Calculate the difference in days
day = (datetime.now() - start_date).days

In [158]:
df = getdata(ticker, timeframe, day)

In [159]:
df

,Open,High,Low,Close,Volume
timestamp,,,,,
2019-09-08,10000.00,10412.65,10000.00,10391.63,3096.291
2019-09-09,10316.62,10475.54,10077.22,10307.00,14824.373
2019-09-10,10307.00,10382.97,9940.87,10102.02,9068.955
2019-09-11,10094.27,10293.11,9884.31,10159.55,10897.922
2019-09-12,10163.06,10450.13,10042.12,10415.13,15609.634
...,...,...,...,...,...
2023-10-25,33917.00,35154.30,33660.60,34487.70,465367.144
2023-10-26,34487.80,34828.70,33750.00,34154.00,339552.964
2023-10-27,34154.00,34253.30,33350.00,33893.10,278052.978


In [34]:
sym_quote = ticker[-4:]
sym_quote

'USDT'

In [35]:
exchange.fetch_balance()['free'][sym_quote]

10000.0